In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.widgets import Slider
%matplotlib notebook

from tbp.monty.frameworks.utils.logging_utils import (load_stats,
                                                         print_overall_stats,
                                                         print_unsupervised_stats)
from tbp.monty.frameworks.utils.plot_utils import (plot_graph,
                                                         show_initial_hypotheses, 
                                                         plot_evidence_at_step,)

from tbp.monty.frameworks.utils.spatial_arithmetics import (non_singular_mat,
                                                              get_right_hand_angle)
from tbp.monty.frameworks.utils.sensor_processing import (get_center_point_normal_v2,
                                                              get_curvature_at_point_v2,)

In [ ]:
pretrain_path = os.path.expanduser("~/tbp/results/monty/pretrained_models/")
pretrained_dict = pretrain_path + "pretrained_ycb_v3/supervised_pre_training_base/pretrained/"

log_path = os.path.expanduser("~/tbp/results/monty/projects/evidence_eval_runs/logs/")

exp_name = "randrot_noise_10distobj_touch/"
exp_path = log_path + exp_name

save_path = exp_path + '/stepwise_examples/'

train_stats, eval_stats, detailed_stats, lm_models = load_stats(exp_path,
                                                                load_train=False,
                                                                load_eval=True,
                                                                load_detailed=True,
                                                                load_models=False,
                                                                pretrained_dict=pretrained_dict,
                                                               )

**Investigate point-normal estimate**

In [ ]:
def get_center_point_normal_v2(point_cloud_base, center_id, neighbor_radius=10):
    """
    :params: point cloud: graph point position in cartesian coordinates (assumes that points haven't
                          been filtered based on whether they lie on an object or not)
    :params: center_id: id of the center point in point_cloud.
    :params: neighbor_radius: Radius around the center whithin which to consider a neighboring pixel.
    :return 
    """
    point_cloud = point_cloud_base.copy()
    if point_cloud[center_id, 3] > 0:
        # Make sure point positions are expressed relative to the center point
        point_cloud[:, :3] -= point_cloud[center_id, :3]

        # Find the center point coordinates in the 2D (64x64) grid
        n_points = point_cloud.shape[0]
        point_idx = np.arange(n_points).reshape(64, 64)
        i, j = np.meshgrid(np.arange(64), np.arange(64))
        idx_array = np.dstack((i, j))
        center_point_idx = idx_array[point_idx == center_id]

        # Compute the relative index position of all points in the patch relative to the center point
        # Distance here is in terms of index units, not in cartesian coordinates.
        idx_rel = idx_array - center_point_idx
        is_neighbor = (np.linalg.norm(idx_rel, axis=2) <= neighbor_radius)
        neighbors_idx = is_neighbor.reshape((n_points,))
        center_neighbors = point_cloud[neighbors_idx, :3]
        neighbors_on_obj = point_cloud[neighbors_idx, 3] > 0

        # Extract neighbors not on object
        center_neighbors_not_on_obj = center_neighbors[~neighbors_on_obj, :].copy()

        # Filter out points that do not lie on an object
        center_neighbors = center_neighbors[neighbors_on_obj, :]

        # Extract nonneighbors
        nonneighbors_pos = point_cloud[~neighbors_idx, :3]

        # Solve linear least-square regression: X^{T}X w = X^{T}y <==> Aw = b
        # TODO: Note that this assumes the independent variables (columns of X) are noise-free, which is not 
        # the case here. A more adequate solution would be a total least square regression which also 
        # account for noise in the independent variables, but is generally trickier to implement.
        X = center_neighbors.copy()
        X[:, 2] = np.ones((center_neighbors.shape[0],))
        y = center_neighbors[:, 2]

        A = np.matmul(X.T, X)
        b = np.matmul(X.T, y)

        valid_pn = True
        if non_singular_mat(A):
            w = np.linalg.solve(A, b)

            # Compute surface normal from fitted weights and normalize it
            point_normal = np.ones((3,))
            point_normal[:2] = -w[:2].copy()
            point_normal = point_normal / np.linalg.norm(point_normal)
                
            # Make sure normal points in viewing direction
            on_obj_mask = (point_cloud[:, 3] > 0).reshape(64, 64)
            neighbor_on_obj_mask = np.multiply(on_obj_mask, is_neighbor)[:, :, np.newaxis]
            neighbor_on_obj_idx = np.multiply(idx_array, neighbor_on_obj_mask)

            # Compute extreme points upon which to compute right and up vectors for viewing dir computation
            r_id_2d = np.unravel_index(neighbor_on_obj_idx[:, :, 0].argmax(keepdims=True)[0], neighbor_on_obj_idx.shape[:2])
            l_id_2d = np.unravel_index(neighbor_on_obj_idx[:, :, 0].argmin(keepdims=True)[0], neighbor_on_obj_idx.shape[:2])
            t_id_2d = np.unravel_index(neighbor_on_obj_idx[:, :, 1].argmax(keepdims=True)[0], neighbor_on_obj_idx.shape[:2])
            b_id_2d = np.unravel_index(neighbor_on_obj_idx[:, :, 1].argmin(keepdims=True)[0], neighbor_on_obj_idx.shape[:2])

            r_id = point_idx[r_id_2d]
            l_id = point_idx[l_id_2d]
            t_id = point_idx[t_id_2d]
            b_id = point_idx[b_id_2d]

            vec_right = point_cloud[r_id, :3] - point_cloud[l_id, :3]
            vec_up = point_cloud[t_id, :3] - point_cloud[b_id, :3]

            view_dir = np.cross(vec_up, vec_right)

            if np.dot(view_dir, point_normal) < 0:
                point_normal *= -1
                
        else: # Not enough point to compute 
            point_normal = np.array([0, 0, 1])
            valid_pn = False
            print(
                    "Warning : Singular matrix encountered in get_center_point_normal_v2!"
            )
            w, center_neighbors, center_neighbors_not_on_obj, nonneighbors_pos = None, None, None, None
    else: # Patch center is not on an object 
        point_normal = np.array([0, 0, 1])
        valid_pn = False
        print(
                "Warning : Singular matrix encountered in get_center_point_normal_v2!"
        )
        w, center_neighbors, center_neighbors_not_on_obj, nonneighbors_pos = None, None, None, None
        
    return point_normal, valid_pn, w, center_neighbors, center_neighbors_not_on_obj, nonneighbors_pos

In [ ]:
# Plot a patch
patch_id = 0
patch_data = np.array(detailed_stats['0']['SM_0']['raw_observations'][patch_id]['semantic_3d'])
pos = patch_data[:, :3]
on_obj = (patch_data[:, 3] > 0).astype(int)

# Plot patch
colormap = np.array(['k', 'r'])
fig = plt.figure(1)
ax = fig.add_subplot(121, projection='3d')
ax.scatter(pos[:,0], pos[:,1], pos[:,2], marker=".", c=colormap[on_obj], alpha=0.25) # neighbors only
ax.set_aspect('equal')
plt.show()

# Chose patch to plot
point_pos = np.array(detailed_stats['0']['SM_0']['raw_observations'][patch_id]['semantic_3d'])

# Calculate center ID for flat semantic obs
obs_dim = int(np.sqrt(point_pos.shape[0]))
half_obs_dim = obs_dim // 2
center_id = half_obs_dim + obs_dim * half_obs_dim
neighbor_radius = 10

# Run function
n_dir, valid_pn, w, neighbors_pos, neighbors_not_on_obj, nonneighbors_pos = get_center_point_normal_v2(point_pos, center_id, neighbor_radius=neighbor_radius)
print("Valid PN: ", valid_pn)

if w is not None:
    # Rescale point_pos
    point_pos[:, :3] -= point_pos[center_id, :3]

    # Retrieve axis limits
    xlim = [np.min(point_pos[:,0]), np.max(point_pos[:,0])]
    ylim = [np.min(point_pos[:,1]), np.max(point_pos[:,1])]
    zlim = [np.min(point_pos[:,2]), np.max(point_pos[:,2])]

    # Define patch surface
    xx = point_pos[:,0].reshape((64, 64))
    yy = point_pos[:,1].reshape((64, 64))
    # xx, yy = np.meshgrid(np.linspace(xlim[0], xlim[1], 100), np.linspace(ylim[0], ylim[1], 100))
    zz = w[0] * xx + w[1] * yy + w[2]

    # Rescale point normal vector for plotting
    pn_vec = 0.25 * np.abs(np.diff(zlim)) * n_dir

    # Plot patch surface, neighbor region, point normal
    fig = plt.figure(1)
    ax = fig.add_subplot(122, projection='3d')
    ax.scatter(neighbors_pos[:,0], neighbors_pos[:,1], neighbors_pos[:,2], marker=".", color="r", alpha=0.25) # neighbors on object only
    ax.scatter(neighbors_not_on_obj[:,0], neighbors_not_on_obj[:,1], neighbors_not_on_obj[:,2], marker=".", color="y", alpha=0.25) # neighbors only
    ax.scatter(nonneighbors_pos[:,0], nonneighbors_pos[:,1], nonneighbors_pos[:,2], marker=".", color="b", alpha=0.1) # non-neighbors only
    ax.plot_surface(xx, yy, zz, alpha=0.25, color="g")
    ax.quiver(0, 0, w[2], pn_vec[0], pn_vec[1], pn_vec[2], color="k")
    ax.set_aspect('equal')
    plt.show()

**Investigate curvature estimate**

In [ ]:
def get_weight_matrix_v2(point_pos, center_id, sigma=10):
    # Extract center and all its neighbors
    n_points = point_pos.shape[0]
    
    # Find indices of center in 64x64 grid
    point_idx = np.arange(n_points).reshape(64, 64)
    x, y = np.meshgrid(np.arange(64), np.arange(64))
    xy = np.dstack((x, y))
    center_pos = xy[point_idx == center_id]
    
    # Compute index distance of all pixels relative to center
    xy_rel = xy - center_pos # Express all points with respect to center point
    point_dist = np.linalg.norm(xy_rel, axis=2)
    
    # Compute weight matrix based on distance
    w_coefs = 1.0 / (np.sqrt(2 * np.pi) * sigma) * np.exp(- np.square(point_dist) / (2 * sigma**2))
    
    fig = plt.figure(2)
    ax = fig.add_subplot(121)
    ax.matshow(w_coefs, cmap=plt.cm.Blues)
    plt.show()
    
    w_diag = w_coefs.reshape((n_points,1))
    w_diag /= np.sum(w_diag)
    return w_diag

In [ ]:
def get_curvature_at_point_v2(point_cloud_base, center_id, n_dir):
    """Compute principal curvatures from point cloud.

    Computes the two principal curvatures of a 2D surface and corresponding
    principal directions

    Arguments
        :param points: point cloud (2d numpy array) based on which the 2D
                surface is approximated
        :param center_id: center point around which the local curvature is
                estimated
        :param normal: surface normal at the center point

    Returns
        k1:     first principal curvature
        k2:     second principal curvature
        dir1:   first principal direction
        dir2:   second principal direction
    """
    point_cloud = point_cloud_base.copy()
    if point_cloud[center_id, 3] > 0:
        # Make sure point positions are expressed relative to the center point
        point_cloud[:, :3] -= point_cloud[center_id, :3]

        # Retrieve weight matrix for weighted least-square regression
        W = get_weight_matrix_v2(point_cloud, center_id, sigma=20)

        # Filter out points that are not on the object
        on_obj = point_cloud[:, 3] > 0
        point_cloud = point_cloud[on_obj, :3]
        W = W[on_obj, :]

        # find two directions u_dir and v_dir orthogonal to point normal (defined by n_dir):
        # If n_dir's z coef is 0 then normal is pointing in (x,y) plane i.e. z dir is orthogonal to point normal
        u_dir = np.array([1, 0, -n_dir[0]/n_dir[2]]) if n_dir[2] else np.array([0, 0, 1])
        u_dir /= np.linalg.norm(u_dir)

        v_dir = np.cross(n_dir, u_dir)
        v_dir /= np.linalg.norm(v_dir)

        # Project point coordinates onto local reference frame
        u = np.matmul(point_cloud, u_dir)
        v = np.matmul(point_cloud, v_dir)
        n = np.matmul(point_cloud, n_dir)

        # Compute basis functions for quadratic regression
        # n = a * u^2 + b * v^2 + c * u * v + d * u + e * v + d * 1
        X = np.zeros((len(point_cloud), 6))
        X[:, 0] = np.multiply(u, u)
        X[:, 1] = np.multiply(v, v)
        X[:, 2] = np.multiply(u, v)
        X[:, 3] = u
        X[:, 4] = v
        X[:, 5] = np.ones(u.shape)

        # Quadratic regression comes down to solving a linear system Aw = b <==> X^{T}WX w = X^{T}Wn
        # Note: matrix multiplication w/ a diagonal matrix is highly-inefficient computationally
        A = np.matmul(X.T, np.multiply(W, X))
        b = np.matmul(X.T, np.multiply(W, n[:, np.newaxis]))

        # Rarely, "a" can be singular, causing numpy to throw an error; appears
        # to be caused by touch-sensor gathering observations that are largely off the
        # object, but not entirely (e.g. <25% visible), resulting in a system
        # with insufficient data to be solvable
        if non_singular_mat(A):
            # Step 2) do least-squares fit to get the parameters of the quadratic form
            params = np.linalg.solve(A, b)

            # Step 3) compute 1st and 2nd fundamental forms guv and buv:
            # TODO: Extract improved point normal estimate from fitted curve
            guv = np.zeros((2, 2))
            guv[0, 0] = 1 + params[3] * params[3]
            guv[0, 1] = params[3] * params[4]
            guv[1, 0] = guv[0, 1]
            guv[1, 1] = 1 + params[4] * params[4]

            buv = np.zeros((2, 2))
            buv[0, 0] = 2 * params[0]
            buv[0, 1] = params[2]
            buv[1, 0] = buv[0, 1]
            buv[1, 1] = 2 * params[1]

            # Step 4) compute the principle curvatures and directions:
            # TODO: here convex PCs are negative but I think they should be positive
            m = np.linalg.inv(guv).dot(buv)
            eigval, eigvec = np.linalg.eig(m)
            idx = eigval.argsort()[::-1]
            eigval_sorted = eigval[idx]
            eigvec_sorted = eigvec[:, idx]

            k1 = eigval_sorted[0]
            k2 = eigval_sorted[1]

            # TODO: sometimes dir1 and dir2 are not orthogonal, why?
            # principal directions in the same coordinate frame as points:
            pc1_dir = eigvec_sorted[0, 0] * u_dir + eigvec_sorted[1, 0] * v_dir
            pc2_dir = eigvec_sorted[0, 1] * u_dir + eigvec_sorted[1, 1] * v_dir
            if get_right_hand_angle(pc1_dir, pc2_dir, n_dir) < 0:
                # always have dir2 point to the righthand side of dir1
                pc2_dir = -pc2_dir
            valid_pc = True

        else:
            k1, k2, pc1_dir, pc2_dir = 0, 0, [0, 0, 0], [0, 0, 0]
            valid_pc = False
            print(
                "Warning : Singular matrix encountered in get-curvature-at-point!"
            )
            u, v, n, params = None, None, None, None

    else:
        k1, k2, pc1_dir, pc2_dir = 0, 0, [0, 0, 0], [0, 0, 0]
        valid_pc = False
        print(
                "Warning : Patch is not centered on object!"
        )
        u, v, n, params = None, None, None, None

    return k1, k2, pc1_dir, pc2_dir, valid_pc, u, v, n, params

In [ ]:
# Run curvature estimate function on point cloud
k1, k2, pc1_dir, pc2_dir, valid_pc, u, v, n, params = get_curvature_at_point_v2(point_pos, center_id, n_dir)

if u is not None:
    # Retrieve axis limits
    ulim = [np.min(u), np.max(u)]
    vlim = [np.min(v), np.max(v)]

    # Define patch surface
    uu, vv = np.meshgrid(np.linspace(ulim[0], ulim[1], 100), np.linspace(vlim[0], vlim[1], 100))

    X = np.zeros((uu.shape[0], vv.shape[1], 6))
    X[:,:,0] = np.multiply(uu, uu)
    X[:,:,1] = np.multiply(vv, vv)
    X[:,:,2] = np.multiply(uu, vv)
    X[:,:,3] = uu
    X[:,:,4] = vv
    X[:,:,5] = np.ones(uu.shape)

    nn = np.matmul(X, params)[:, :, 0]

    fig = plt.figure(2)
    ax = fig.add_subplot(122, projection='3d')
    ax.scatter(u, v, n, marker=".", color="r", alpha=0.25)
    ax.plot_surface(uu, vv, nn, alpha=0.25, color="g")
    ax.set_aspect('equal')
    plt.show()